In [1]:
import pandas as pd
import os
from path import Path
import numpy as np

In [2]:
reviews_df = pd.read_csv('/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv')
#reviews_df = reviews_df.dropna(subset=["review_content"])
movie_id_cols = [
                "rotten_tomatoes_link",
                "movie_title",
                "release_year",
            ]
#reviews_df["rotten_tomatoes_link"].nunique()

In [5]:
reviews_df = reviews_df.dropna(subset=['rating_normalized'])

In [18]:
reviews_df[reviews_df['original_release_date'].isna()]

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,movie_title,movie_info,...,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,imdb_id,imdb_rating,awards,box_office,rating_normalized


In [21]:
reviews_df["release_year"] = pd.to_datetime(reviews_df["original_release_date"]).dt.year
len(pd.unique(reviews_df[["movie_title", "release_year"]].values.ravel('K')))

8009

In [7]:
len(reviews_df)/8075

68.37919504643963

In [52]:
reviews_df['critic_name'].nunique()

6127

In [53]:
reviews_df.groupby('critic_name').size().mean()

np.float64(89.4759262281704)

In [54]:
reviews_per_movie = reviews_df.groupby('rotten_tomatoes_link').size()
reviews_per_movie.mean()

np.float64(68.37919504643963)

In [55]:
reviews_per_movie

rotten_tomatoes_link
m/0814255                                121
m/0878835                                100
m/10                                      12
m/1000013-12_angry_men                    34
m/1000079-20000_leagues_under_the_sea     13
                                        ... 
m/zoolander                               80
m/zoolander_2                            170
m/zoom_2006                               57
m/zootopia                               207
m/zulu                                    10
Length: 8075, dtype: int64

8075 movies. 94.4 review (and reviewer) per movie.
9825 reviewers.  78.5 reviews (and movies) per reviewer on avg.

In [56]:
reviews_df.groupby('rotten_tomatoes_link')['critic_name'].nunique().mean()

np.float64(67.64730650154799)

In [57]:
reviews_df.groupby('critic_name')['rotten_tomatoes_link'].nunique().mean()

np.float64(89.1548881997715)

In [4]:
# Apply normalization
reviews_df['rating_normalized'] = reviews_df['review_score'].apply(normalize_rating)

In [48]:
# Statistics
print("="*60)
print("RATING FORMAT ANALYSIS")
print("="*60)

print("\n1. Sample original ratings:")
print(reviews_df['review_score'].value_counts().head(20))

print("\n2. Normalized ratings stats:")
print(reviews_df['rating_normalized'].describe())

print(f"\n3. Valid ratings: {reviews_df['rating_normalized'].notna().sum()} / {len(reviews_df)}")
print(f"   Success rate: {reviews_df['rating_normalized'].notna().sum() / len(reviews_df) * 100:.1f}%")

print("\n4. Rating format breakdown:")
# Detect format types
def detect_format(rating_str):
    if pd.isna(rating_str) or rating_str == '<null>':
        return 'null'
    rating_str = str(rating_str).strip().replace(" ", "")
    if '/' in rating_str:
        return 'fraction'
    if rating_str.upper() in ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']:
        return 'letter'
    try:
        float(rating_str)
        return 'numeric'
    except:
        return 'unknown'

reviews_df['format_type'] = reviews_df['review_score'].apply(detect_format)
print(reviews_df['format_type'].value_counts())

print("\n5. Sample unknown formats:")
unknown = reviews_df[reviews_df['format_type'] == 'unknown']['review_score'].value_counts().head(10)
if len(unknown) > 0:
    print(unknown)
else:
    print("None!")

RATING FORMAT ANALYSIS

1. Sample original ratings:
review_score
3/5      53502
4/5      52168
3/4      51257
2/5      35719
2/4      34393
2.5/4    32641
3.5/4    25157
3.5/5    20800
B        16683
5/5      15354
1.5/4    14795
2.5/5    14637
B+       13795
4/4      13529
1/5      12233
B-       10756
1/4      10662
C        10163
4.5/5     8682
7/10      8169
Name: count, dtype: int64

2. Normalized ratings stats:
count    552162.000000
mean          6.533300
std           2.088468
min           0.000000
25%           5.000000
50%           7.000000
75%           8.000000
max          90.000000
Name: rating_normalized, dtype: float64

3. Valid ratings: 552162 / 762263
   Success rate: 72.4%

4. Rating format breakdown:
format_type
fraction    460682
null        210098
letter       90358
numeric       1125
Name: count, dtype: int64

5. Sample unknown formats:
None!
